Versione CUDA: 12.1
Versione pytorch: 2.3.0+cu121
Versione torchvision: 0.18.0+cu121

(pytorch e torchvision possono essere installate con il seguente comando da terminale:
"pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121")

In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.3.0+cu121
0.18.0+cu121


In [1]:

import torch
 
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
 
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID of current CUDA device:{torch.cuda.current_device()}")
       
print(f"Name of current CUDA device:{torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? True
CUDA version: 12.1
ID of current CUDA device:0
Name of current CUDA device:NVIDIA GeForce MX230


Il seguente modello è quello di default di YOLO, e lo uso per fare il tracking dei giocatori:

In [2]:
from ultralytics import YOLO
import cv2



# Apri il video di input
cap = cv2.VideoCapture('input.mp4')  # Sostituisci con il percorso del tuo video

# Ottieni le proprietà del video di input
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Configura il writer per il video di output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Usa il codec mp4v per il formato MP4
out = cv2.VideoWriter('output_videos/output_video.mp4', fourcc, 24, (width, height))

# Esegui inferenza su un video con stream=True
model = YOLO('models/yolov8x.pt')

results = model.predict("input.mp4", stream=True, device='cuda')

# Processa i risultati e scrivi i frame nel video di output
for result in results:
    # Ottieni il frame originale
    frame = result.orig_img

    # Ottieni il frame con le annotazioni
    annotated_frame = result.plot()
    # La funzione plot() è una funzione fornita dall'oggetto risultato restituito dalla funzione predict() di YOLO.
    #La funzione plot() prende i risultati dell'inferenza (come i bounding box delle rilevazioni degli oggetti, 
    #le etichette degli oggetti, ecc.) e disegna queste annotazioni sul frame originale.

    # Scrivi il frame annotato nel video di output
    out.write(annotated_frame)

# Rilascia le risorse
cap.release()
out.release()

print("Il video annotato è stato salvato in output_video.mp4")


video 1/1 (1/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 8 persons, 1 sports ball, 2 tennis rackets, 1 chair, 1 clock, 31.3ms
video 1/1 (2/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 8 persons, 1 tennis racket, 1 chair, 1 clock, 29.9ms
video 1/1 (3/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 8 persons, 1 sports ball, 2 tennis rackets, 1 chair, 1 clock, 29.9ms
video 1/1 (4/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 8 persons, 1 sports ball, 1 tennis racket, 1 chair, 1 clock, 27.8ms
video 1/1 (5/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 8 persons, 1 sports ball, 2 tennis rackets, 1 chair, 1 clock, 28.3ms
video 1/1 (6/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 8 persons, 1 sports ball, 3 tennis rackets, 1 chair, 1 clock, 25.2ms
video 1/1 (7/1994) d:\Linux\Universit\

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


Questo modello invece è quello da me addestrato per fare la detection della pallina da tennis:

In [4]:
from ultralytics import YOLO

model = YOLO('models/model_tennis_ball_detection.pt')

result = model.predict("input.mp4", conf=0.2, save=True)



video 1/1 (frame 1/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 (no detections), 112.2ms
<generator object BasePredictor.stream_inference at 0x0000025F17AEAB00>
video 1/1 (frame 2/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 (no detections), 39.9ms
<generator object BasePredictor.stream_inference at 0x0000025F17AEAB00>
video 1/1 (frame 3/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 1 tennis ball, 46.9ms
<generator object BasePredictor.stream_inference at 0x0000025F17AEAB00>
video 1/1 (frame 4/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 1 tennis ball, 41.7ms
<generator object BasePredictor.stream_inference at 0x0000025F17AEAB00>
video 1/1 (frame 5/1994) d:\Linux\Universit\Anno III\Lab_IA\Tesi\Tennis-Analyzer\input.mp4: 384x640 1 tennis ball, 49.5ms
<generator object BasePredictor.stream_inference at 0x0000025F17AEAB00>
video 1/1 (frame 6/1994)

KeyboardInterrupt: 